# Determine Insurance Premiums

This notebook aims to predict the value of insurance premiums based on customer informations.

When you finish this notebook, you'll have a foundational knowledge on Predictive Analysis

### Problem Statement

- Given a set amount of features, treat them to extract the best possible model to predict the final value of the insurance premium

- The dataset was extracted from kaggle https://www.kaggle.com/competitions/playground-series-s4e12/data
    - This dataset was generated from a similar dataset https://www.kaggle.com/datasets/schran/insurance-premium-prediction
    - They are just similar and can be used in conjunction, but they are not the same

### Evaluation

- Submissions are evaluated using the Root Mean Squared Logarithmic Error (RMSLE) 
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_log_error.html#sklearn.metrics.root_mean_squared_log_error

    - Advantages of using RMLSE:
        - https://medium.com/analytics-vidhya/root-mean-square-log-error-rmse-vs-rmlse-935c6cc1802a
        - Robustness to the effect of outliers
        - The internal part of the RMSLE is fundamentally a calculation of relative error between the predicted and the actual values
        - Biased penalty - RMSLE incurs a larger penalty for the underestimation of the Actual variable than the Overestimation. More penalty is incurred when the predicted Value is less than the Actual Value
            - Especially useful for business cases where the underestimation of the target variable is no acceptable but overestimation can be tolerated
                
                **In this case, RMLSE is very useful since the Insurance Premium is the value paid to maintain coverage**
                
                **If we underestimate the premium to be paid, the insurance company will be loosing money. But we won't have much problem if the premium is slightly overestimated**

### Submission file

For each id row in the test set, you must predict the continuous target Premium Amount. The file should contain a header and have the following format:

id,Premium Amount

1200000,1102.545

1200001,1102.545

1200002,1102.545

etc.

#### Data loading

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import seaborn as sns

In [2]:
#Import train set
df_path = 'depression_train.csv'
df = pd.read_csv(df_path,index_col="id")

df.head()

#Import test set
df_test_path = 'depression_test.csv'
df_test = pd.read_csv(df_test_path,index_col="id")

X_test = df_test.copy()

In [3]:
#Separate target from train set
#X_train.dropna(axis=0,subset=['Depression'],inplace=True)
#y_train = X_train.pop('Depression')

In [4]:
#Mark train column as 'True' for train set and 'False' for test set
#So I can join both sets and make necessary transformations on both sets at the same time
df['is_train'] = 1
X_test['is_train'] = 0

df_total = pd.concat([df, X_test])

In [5]:
df_total.head()

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression,is_train
id,,,,,,,,,,,,,,,,,,,,
0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0.0,1
1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1.0,1
2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1.0,1
3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1.0,1
4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0.0,1


In [6]:
df_total.tail()

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression,is_train
id,,,,,,,,,,,,,,,,,,,,
234495,Zoya,Female,49.0,Jaipur,Working Professional,Pilot,NaN,3.0,NaN,NaN,5.0,Less than 5 hours,Moderate,BSc,Yes,2.0,2.0,Yes,NaN,0
234496,Shlok,Male,29.0,Ahmedabad,Working Professional,Pilot,NaN,5.0,NaN,NaN,1.0,7-8 hours,Moderate,BE,Yes,11.0,3.0,Yes,NaN,0
234497,Rishi,Male,24.0,Visakhapatnam,Student,NaN,1.0,NaN,7.51,4.0,NaN,7-8 hours,Moderate,B.Tech,No,7.0,1.0,No,NaN,0
234498,Eshita,Female,23.0,Kalyan,Working Professional,Marketing Manager,NaN,4.0,NaN,NaN,2.0,5-6 hours,Healthy,BA,Yes,7.0,5.0,Yes,NaN,0
234499,Gauri,Female,43.0,Varanasi,Working Professional,Educational Consultant,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,B.Ed,No,11.0,2.0,No,NaN,0


### Data understanding

In [10]:
df_total.dtypes

Name                                      object
Gender                                    object
Age                                      float64
City                                      object
Working Professional or Student           object
Profession                                object
Academic Pressure                        float64
Work Pressure                            float64
CGPA                                     float64
Study Satisfaction                       float64
Job Satisfaction                         float64
Sleep Duration                            object
Dietary Habits                            object
Degree                                    object
Have you ever had suicidal thoughts ?     object
Work/Study Hours                         float64
Financial Stress                         float64
Family History of Mental Illness          object
Depression                               float64
is_train                                   int64
dtype: object

In [58]:
feature_check = df_total['Family History of Mental Illness']

In [59]:
#Show unique values
print(feature_check.unique())

['No' 'Yes']


In [60]:
#Count unique values
print(feature_check.nunique())

2


In [61]:
#Count the occurance of each unique value in the column - check crazy values
print(feature_check.value_counts())

Family History of Mental Illness
No     117876
Yes    116624
Name: count, dtype: int64


In [62]:
#Number of missing values in each column of data
missing_val_count_by_column = (df_total.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Profession             61262
Academic Pressure     187836
Work Pressure          46696
CGPA                  187836
Study Satisfaction    187836
Job Satisfaction       46684
Dietary Habits             9
Degree                     4
Financial Stress           4
Depression             93800
dtype: int64


**Features - First notes**

- Name: Name of the insured individual (Categorical)
- Gender: Gender of the insured individual (Categorical: Male, Female)
- Age: Age of the insured individual (Numerical)
- City: City of the insured individual (Categorical: 124 unique values)
- Working Professional or Student (Categorical: Working Professional, Student)
- Profession: Profession of the insured individual (Categorical: 81 unique values **missing values**)
- Academic Pressure: Leve of academic pressure (Numerical: 1-5 **missing values**) - *Missing values are for "Working Professionals"?* 
- Work Pressure: Level of work pressure (Numerical: 1-5 **missing values**) - *Missing values are for "Studentes"?*
    - Academic Pressure and Work Pressure might be combined into Feature **Pressure**
- CGPA: Cumulative Grade Point Average - Student related (Numerical: 0-10) **Missing for working professionals**
- Study Satisfaction: Level of study satisfaction (Numerical: 1-5) **Missing values** 
- Job Satisfaction: Level of working satisfaction (Numerical: 1-5) **Missing values**
    - Study satisfaction and Job satisfaction might be combined into Feature **Satisfaction**
- Sleep duration: Range of sleep duration (Categorical: *doesn't seem useful*) - **Check on crazy values**
- Dietary Habits: Category of heathiness (Categorical: Healthy, Unhealthy, Moderate) - **Check on crazy values**
- Degree: Type of degree (Categorical: 156 unique values - *Hardly useful*)
- Have you ever had suicidal thoughts ?: Yes/No question (Categorical: Yes/No)
- Financial Stress: Level of financial stress (Numerical 1-5) **Missing Values**
- Family History of Mental Illness: Yes/No question (Categorical: Yes/No)

### Data cleaning

- Profession
- CGPA
- Sleep Duration
- Dietary Habits
- Financial Stress

(Ps: Pressure and Satisfaction will be treated on *Feature Engineering*)

In [71]:
df_total[['Working Professional or Student','Profession']].head(20)

,Working Professional or Student,Profession
id,,
0,Working Professional,Chef
1,Working Professional,Teacher
2,Student,Student
3,Working Professional,Teacher
4,Working Professional,Business Analyst
5,Working Professional,Finanancial Analyst
6,Working Professional,Chemist
7,Working Professional,Teacher
8,Student,Student


In [70]:
# Filter rows where column 'Profession' has NaN values
filtered_df = df_total[df_total['Profession'].isna()]


# Count unique values in column 'A' for these rows
unique_count = filtered_df['Working Professional or Student'].unique()
print(unique_count)

[]


For 'Profession' there are missing values for 'Student' and for 'Working Professional'. In the first case 'Profession' will be filled with 'Student' in the seccond case 'Unknown'

In [69]:
# Fill column 'Profession' based on values for 'Working Professional or Student'
df_total['Profession'] = np.where(df_total['Profession'].isna() & (df_total['Working Professional or Student'] == 'Working Professional'), 'Unknown', df_total['Profession'])
df_total['Profession'] = np.where(df_total['Profession'].isna() & (df_total['Working Professional or Student'] == 'Student'), 'Student', df_total['Profession'])